<a href="https://colab.research.google.com/github/janShi1105/science/blob/main/M_L14_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20
steps_per_epoch = np.ceil(60000/ BATCH_SIZE)


In [2]:
def preprocess(item):
  image = item['image']
  label = item['label']
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.reshape(image, (-1,))
  return {'image-pixels': image}, label[..., tf.newaxis]

In [7]:
def train_input_fn():
  datasets = tfds.load(name='mnist')
  mnist_train = datasets['train']
  dataset = mnist_train.map(preprocess)
  dataset = dataset.shuffle(BUFFER_SIZE)
  dataset = dataset.batch(BATCH_SIZE)
  return dataset.repeat()

def eval_input_fn():
  datasets = tfds.load(name='mnist')
  mnist_test = datasets['test']
  dataset = mnist_test.map(preprocess).batch(BATCH_SIZE)
  return dataset

In [8]:
image_feature_column = tf.feature_column.numeric_column(key='image-pixels', shape=(28*28))

In [9]:
dnn_classifier = tf.estimator.DNNClassifier(feature_columns=[image_feature_column], hidden_units=[32,16], n_classes=10, model_dir='models/mnist-dnn/')

In [10]:
dnn_classifier.train(input_fn=train_input_fn, steps=NUM_EPOCHS * steps_per_epoch)
eval_result = dnn_classifier.evaluate(input_fn=eval_input_fn)
print(eval_result)

Instructions for updating:
Use standard file utilities to get mtimes.


{'accuracy': 0.8985, 'average_loss': 0.36893693, 'loss': 0.3687791, 'global_step': 37520}


In [11]:
tf.random.set_seed(1)
np.random.seed(1)
x = np.random.uniform(low=-1, high=1, size=(200,2))
y = np.ones(len(x))
y[x[:,0] * x[:,1]<0] = 0
x_train = x[:100, :]
y_train = y[:100]
x_valid = x[100:, :]
y_valid = y[100:]

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,), name='input-features'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'), 
    tf.keras.layers.Dense(units=4, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

In [13]:
def train_input_fn(x_train,  y_train, batch_size=8):
  dataset = tf.data.Dataset.from_tensor_slices(({'input-features': x_train}, y_train.reshape(-1,1)))
  return dataset.shuffle(100).repeat().batch(batch_size)

def eval_input_fn(x_test, y_test=None, batch_size=8):
  if y_test is None:
    dataset = tf.data.Dataset.from_tensor_slices({'input-features': x_test})
  else:
    dataset = tf.data.Dataset.from_tensor_slices(({'input-features': x_test}, y_test.reshape(-1,1)))

    return dataset.batch(batch_size)

features = [tf.feature_column.numeric_column(key='input-features:', shape=(2,))]

In [14]:
model.compile(optimizer=tf.keras.optimizers.SGD(), loss=tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy()])
my_estimator = tf.keras.estimator.model_to_estimator(keras_model=model, model_dir = 'models/estimator-for-XOR/')

/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [15]:
num_epochs = 200
batch_size = 2
steps_per_epoch = np.ceil(len(x_train) / batch_size)
my_estimator.train(input_fn=lambda: train_input_fn(x_train, y_train, batch_size), steps=num_epochs * steps_per_epoch)
my_estimator.evaluate(input_fn = lambda: eval_input_fn(x_valid, y_valid, batch_size))

/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


{'binary_accuracy': 0.97, 'loss': 0.08625225, 'global_step': 10000}